In [1]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
nativesql_jars = "/path/to/gluten-XXXX.jar"
conf = SparkConf().setAppName("PySpark Gluten").setMaster("yarn")
conf.set("spark.executor.instances", "2")
conf.set("spark.executor.memory", "6g")
conf.set("spark.executor.cores", "2")
conf.set("spark.driver.memory", "2g")
conf.set("spark.memory.offHeap.enabled", "true")
conf.set("spark.memory.offHeap.size", "2g")
conf.set("spark.executor.memoryOverhead", "384M")
conf.set("spark.driver.extraClassPath", nativesql_jars)
conf.set("spark.executor.extraClassPath", nativesql_jars)
conf.set("spark.plugins", "org.apache.gluten.GlutenPlugin")
conf.set("spark.gluten.sql.columnar.backend.lib", "velox")
conf.set("spark.gluten.loadLibFromJar", "false")
conf.set("spark.shuffle.manager", "org.apache.spark.shuffle.sort.ColumnarShuffleManager")
sc = SparkContext(conf=conf)
from pyspark.sql import SparkSession
spark_session = SparkSession(sc)


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/root/spark-3.3.1-bin-hadoop2/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/root/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
24/04/03 22:04:56 WARN Utils: Your hostname, vsr542 resolves to a loopback address: 127.0.1.1; using 10.0.2.142 instead (on interface eno1)
24/04/03 22:04:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 22:04:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/0

In [2]:
df = spark_session.createDataFrame(
    [
        ("Normal", "Cellphone", 6000),
        ("Normal", "Tablet", 1500),
        ("Mini", "Tablet", 5500),
        ("Mini", "Cellphone", 5000),
        ("Foldable", "Cellphone", 6500),
        ("Foldable", "Tablet", 2500),
        ("Pro", "Cellphone", 3000),
        ("Pro", "Tablet", 4000),
        ("Pro Max", "Cellphone", 4500)
    ],
    ["product", "category", "revenue"]
)
df.filter("category = 'Cellphone'").explain()

== Physical Plan ==
VeloxColumnarToRowExec
+- ^(1) FilterExecTransformer (isnotnull(category#1) AND (category#1 = Cellphone))
   +- ^(1) InputIteratorTransformer[product#0, category#1, revenue#2L]
      +- ^(1) InputAdapter
         +- ^(1) RowToVeloxColumnar
            +- *(1) Scan ExistingRDD[product#0,category#1,revenue#2L]


